<a href="https://colab.research.google.com/github/jong9810/TensorFlow-2.0/blob/main/10_2_Transfer_Learning_Ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning Example : Cats and Dogs
- Cats and Dogs 데이터 셋은 CNN 아키텍처를 구축하고 평가하기 위한 일종의 Hello World같은 학습 데이터라고 할 수 있다.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

tf.__version__

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# 1. 학습 데이터 불러오기
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

In [ ]:
import os
import shutil

if os.path.exists('/content/cats_and_dogs_filtered/'):
    shutil.rmtree('/content/cats_and_dogs_filtered/')
    print('/content/cats_and_dogs_filtered/  is removed!!!')

In [ ]:
# 압축파일 풀기
import zipfile

with zipfile.ZipFile('/content/cats_and_dogs_filtered.zip', 'r') as target_file:
    target_file.extractall('/content/cats_and_dogs_filtered/')

In [ ]:
# 2. 모델 구축 (pre-trained Xception + User-defined Classifier)
IMG_WIDTH = 224
IMG_HEIGHT = 224

base_model = Xception(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

model = Sequential()

model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(2, activation='softmax')) # 정답이 cat, dog 두 종류이므로 출력층 노드 수 = 2

model.summary()

In [ ]:
# 3. ImageDataGenerator 정의
train_dir = '/content/cats_and_dogs_filtered/cats_and_dogs_filtered/train'
test_dir = '/content/cats_and_dogs_filtered/cats_and_dogs_filtered/validation'

train_data_gen = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1)
test_data_gen = ImageDataGenerator(rescale=1./255) # 이미지 읽어올 때 정규화됨

In [ ]:
train_data = train_data_gen.flow_from_directory(train_dir, batch_size=32, color_mode='rgb', shuffle=True, class_mode='categorical', target_size=(IMG_WIDTH, IMG_HEIGHT))
test_data = test_data_gen.flow_from_directory(test_dir, batch_size=32, color_mode='rgb', shuffle=True, class_mode='categorical', target_size=(IMG_WIDTH, IMG_HEIGHT))

In [ ]:
# 정답 확인
# class_indices 속성을 이용하면, 문자열로 표시되는 데이터의 정답이 어떤 숫자로 매칭되어 있는지 확인할 수 있음.
print(train_data.class_indices.items())
print(test_data.class_indices.items())

In [ ]:
# 4. 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(2e-5), metrics=['accuracy'])
# 전이 학습에서는 학습율을 낮게 설정해서 pre-trained weights를 조금씩 업데이트 해주는 것이 중요하다.

In [ ]:
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# EarlyStoppin 함수를 사용하면 오버피팅이 크기 발생하기 전에 계산을 멈출 수 있다.

In [ ]:
save_file_name = './cats_and_dogs_filtered_Xception_Colab.h5'

checkpoint = ModelCheckpoint(save_file_name, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
earlystopping = EarlyStopping(monitor='val_loss', patience=5)

hist = model.fit(train_data, epochs=30, validation_data=test_data, callbacks=[checkpoint, earlystopping])

In [ ]:
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='validation')
plt.title('Loss Trend')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='validation')
plt.title('Accuracy Trend')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
# 5. 테스트 데이터 로드 및 예측
import os
import shutil

if os.path.exists('/content/test_image_dir/'):
    shutil.rmtree('/content/test_image_dir/')
    print('/content/test_image_dir/  is removed !!!')

In [ ]:
import zipfile

with zipfile.ZipFile('/content/test_image_dir.zip', 'r') as target_file:
    target_file.extractall('/content/test_image_dir/')

In [ ]:
import cv2
import glob

test_img_list = []

test_img_name_list = glob.glob('test_image_dir/*') # 테스트 이미지 파일이름 리스트

# 테스트 이미지 파일 로드 및 전처리 작업
for i in range(len(test_img_name_list)):
    src_img = cv2.imread(test_img_name_list[i], cv2.IMREAD_COLOR) # 이미지 로드
    src_img = cv2.resize(src_img, dsize=(IMG_WIDTH, IMG_HEIGHT))  # 사이즈 변경

    dst_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)

    dst_img = dst_img / 255.0 # 이미지 정규화

    test_img_list.append(dst_img)

In [ ]:
plt.figure(figsize=(8,6))

for i in range(len(test_img_list)):
    plt.subplot(2, 3, i+1)
    plt.axis('off')
    plt.imshow(test_img_list[i])
plt.show()

In [ ]:
for i in range(len(test_img_list)):
    print(test_img_list[i].shape)

In [ ]:
pred = model.predict(np.array(test_img_list))

class_name = ['cat', 'dog'] # ImageDataGenerator 에서 cat = 0, dog = 1

plt.figure(figsize=(8,6))

for i in range(len(pred)):
    plt.subplot(2,3,i+1)
    prediction = str(class_name[np.argmax(pred[i])])
    probability = '{0:0.2f}'.format(100 * max(pred[i]))
    title_str = prediction + ' , ' + probability + '%'
    plt.title(title_str)
    plt.imshow(test_img_list[i])
plt.show()

In [ ]:
# 추가적으로 해보거나 알고 싶은 것
# 1. ModelCheckpoint() 함수에 대해 조사하기
# 2. 왜 전이학습을 할 때 학습율을 작게 해야 하는지 이유 알아보기
# 3. os, zipfile, shutil, cv2, glob 등 파일을 다운 받거나 이동하는 클래스에 대해 더 공부하기
# 4. optimizer = Adam() 이 어떤 것인지?
# 5. softmax, relu 등 활성화 함수에는 어떤 것들이 있는지
# 6. !wget 이 뭔지?
# 7. 